In [2]:
%pip install openai graphrag pandas requests python-dotenv langchain numpy tiktoken matplotlib scikit-learn pyyaml pydantic instructor
from IPython.display import clear_output ; clear_output()

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

is_azure = (
  os.getenv("AZURE_OPENAI_ENDPOINT", default="") != "" and
  os.getenv("OPENAI_API_KEY", default="") == ""
)

GPT_4_O_MODEL_NAME = os.getenv("GPT_4_O_MODEL_NAME", default="gpt-4o")
TEXT_EMBEDDING_3_LARGE_MODEL_NAME = os.getenv("TEXT_EMBEDDING_3_LARGE_MODEL_NAME", default="text-embedding-3-large")

if is_azure:
  AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
  AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
  AZURE_OPENAI_API_VERSION = "2024-05-01-preview"
  from openai import AzureOpenAI
  oai = AzureOpenAI(azure_endpoint=AZURE_OPENAI_ENDPOINT, api_key=AZURE_OPENAI_API_KEY, api_version=AZURE_OPENAI_API_VERSION)
else:
  OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
  from openai import OpenAI
  oai = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
import requests
import os

if not os.path.exists('data'): os.makedirs('data')

if not os.path.exists('data/french_revolution.md'):
  french_revolution = requests.get("https://r.jina.ai/https://en.wikipedia.org/wiki/French_Revolution").text.split('\nSee also')[0]
  with open('data/french_revolution.md', 'w') as f:
    f.write(french_revolution)
else:
  with open('data/french_revolution.md', 'r') as f:
    french_revolution = f.read()

print(french_revolution[:123])

Title: French Revolution

URL Source: https://en.wikipedia.org/wiki/French_Revolution

Published Time: 2001-10-18T00:19:10Z


In [5]:
from langchain.text_splitter import MarkdownTextSplitter
import pandas as pd

if not os.path.exists('data/embeddings.parquet'):
  embeddings = pd.DataFrame(columns=['Topic', 'Text', 'Embedding'])

  splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=250)

  chunks = splitter.split_text(french_revolution)
  chunk_embeddings = oai.embeddings.create(
    input=chunks,
    model=TEXT_EMBEDDING_3_LARGE_MODEL_NAME
  )
  for i, chunk in enumerate(chunks):
    try:
      topic = oai.chat.completions.create(
        model=GPT_4_O_MODEL_NAME,
        messages=[
          {
            "role": "system",
            "content": ("Read the user-provided text carefully and output its topic as a short sentence. "
                        "For example: 'Key events in the life of George Washington', 'Inflation in the Weimar republic'. "
                        "Do not add any additional text such as punctuation, markup, or quoting. Only output the topic.")},
          {"role": "user", "content": chunk}
        ],
        max_tokens=23,
        temperature=0.5,
      ).choices[0].message.content
    except Exception:
      pass
    embeddings.loc[len(embeddings)] = [topic or chunk[:23], chunk, chunk_embeddings.data[i].embedding]
  embeddings.to_parquet('data/embeddings.parquet')
else:
  embeddings = pd.read_parquet('data/embeddings.parquet')

embeddings

,Topic,Text,Embedding
0,French Revolution,Title: French Revolution\n\nURL Source: https:...,"[-0.02485630474984646, 0.0026923343539237976, ..."
1,Key events of the French Revolution,"The Storming of the Bastille, 14 July 1789\n\n...","[-0.005255864467471838, 0.0037901357281953096,..."
2,The French Revolution and its impact on modern...,The French Revolution[a] was a period of polit...,"[-0.027185581624507904, -0.016875529661774635,..."
3,Causes and early events of the French Revolution,Its causes are generally agreed to be a combin...,"[-0.020989766344428062, -0.024809090420603752,..."
4,Key events in the early stages of the French R...,which was converted into a National Assembly i...,"[-0.009496822953224182, -0.017249926924705505,..."
...,...,...,...
296,Alphonse Aulard's contributions to the study o...,evidence.[266][267] Alphonse Aulard (1849–1928...,"[-0.0005632034735754132, 0.021342121064662933,..."
297,Marxist socio-economic analysis of the French ...,Socio-economic analysis and a focus on the exp...,"[-0.003857595380395651, 0.007019456941634417, ..."
298,Alfred Cobban's critique of Jacobin-Marxist in...,Alfred Cobban challenged Jacobin-Marxist socia...,"[0.004117575474083424, 0.00744161382317543, -0..."
299,Political decisions and radicalization during ...,"In their 1965 work, La Revolution française, F...","[0.017132386565208435, 0.012499384582042694, -..."


In [6]:
import numpy as np
import tiktoken

def cosine_similarity(vector1, vector2):
  dot_product = np.dot(vector1, vector2)
  norm1 = np.linalg.norm(vector1)
  norm2 = np.linalg.norm(vector2)
  similarity = dot_product / (norm1 * norm2)
  return similarity

tokenizer = tiktoken.encoding_for_model('gpt-4o')

def embeddings_search(query, max_tokens=10000, k=100, min_similarity=0.2):
  query_embedding = oai.embeddings.create(
    input=[query],
    model=TEXT_EMBEDDING_3_LARGE_MODEL_NAME
  ).data[0].embedding
  results = embeddings.copy()
  results['Similarity'] = results['Embedding'].apply(lambda x: cosine_similarity(x, query_embedding))
  results = results.sort_values(by='Similarity', ascending=False).head(k)
  results = results[results['Similarity'] >= min_similarity]
  results['Tokens'] = results['Text'].apply(lambda txt: len(tokenizer.encode(txt)))
  while results['Tokens'].sum() > max_tokens:
    results = results[:-1]
  return results

In [7]:
DEFAULT_RESPONSE_TYPE = 'Summarize and explain in 1-2 paragraphs with bullet points using at most 300 tokens'
DEFAULT_MAX_CONTEXT_TOKENS = 10000

from graphrag.query.structured_search.global_search.reduce_system_prompt import REDUCE_SYSTEM_PROMPT

def ask_embeddings(query, response_type=DEFAULT_RESPONSE_TYPE):
  results = embeddings_search(query, max_tokens=DEFAULT_MAX_CONTEXT_TOKENS)
  response = oai.chat.completions.create(
    model=GPT_4_O_MODEL_NAME,
    messages=[
      {
        "role": "system",
        "content": REDUCE_SYSTEM_PROMPT.format(
          response_type=response_type,
          report_data="---\n---\n".join(results['Text'].tolist()),
        ),
      },
      {"role": "user", "content": query}
    ],
    max_tokens=4000,
    temperature=0.5,
  ).choices[0].message.content
  return response

In [8]:
from IPython.display import Markdown

result = ask_embeddings('Who was Robespierre and what was his role in the French revolution?')

Markdown(result)

### Maximilien Robespierre: Role in the French Revolution

Maximilien Robespierre was a prominent and polarizing figure during the French Revolution, known for his radical stances and significant influence over revolutionary policies and events. Here are key points about his role:

- **Political Leadership**: Robespierre was a leading member of the radical Montagnard faction and played a crucial role in the Committee of Public Safety, which effectively governed France during the Reign of Terror [Data: Reports (4, 7, 10, +more)].
- **Reign of Terror**: He was instrumental in implementing the Reign of Terror, during which around 16,000 people were executed for counter-revolutionary activities, and another 40,000 may have been summarily executed or died awaiting trial [Data: Reports (7, 17, 20, +more)].
- **Radical Reforms**: Robespierre supported and helped enact radical reforms, such as universal male suffrage and the Law of 22 Prairial, which expedited the trial and execution of perceived enemies of the revolution [Data: Reports (11, 30, +more)].
- **Downfall and Execution**: His downfall came when he was perceived as a dictator by his peers. After refusing to name alleged conspirators against the Republic, he and his allies were arrested and executed on 28 July 1794, marking the end of the Reign of Terror [Data: Reports (1, 7, 29, +more)].

Robespierre's legacy is complex; he is seen by some as a champion of revolutionary ideals and by others as a tyrant responsible for significant bloodshed.

In [9]:
result = ask_embeddings('Timeline of the French revolution')

Markdown(result)

### Timeline of the French Revolution

The French Revolution, spanning from 1789 to 1799, was a decade-long period of profound political and social upheaval in France. Below are key events and milestones:

- **May 5, 1789**: The Estates General convened, marking the beginning of the Revolution.
- **July 14, 1789**: Storming of the Bastille, symbolizing the fall of the Ancien Régime and leading to radical measures, including the abolition of feudalism [Data: Reports (6, 48, 49, 51, 52, +more)].
- **August 26, 1789**: Declaration of the Rights of Man and of the Citizen published, emphasizing individual freedoms and equality [Data: Reports (56, 57, 58)].
- **September 1792**: Proclamation of the French First Republic, following the abolition of the monarchy [Data: Reports (6, 100, 101, 106, 108)].
- **January 21, 1793**: Execution of Louis XVI, signaling the end of the monarchy and escalating the Reign of Terror [Data: Reports (6, 108, 109, 114, 118)].
- **July 1794**: End of the Reign of Terror with the execution of Robespierre [Data: Reports (6, 121, 122)].
- **November 9, 1799**: Coup of 18 Brumaire, leading to the establishment of the French Consulate and marking the end of the Revolution [Data: Reports (6, 158, 159)].

### Implications

- **Political Change**: The Revolution led to the abolition of the Ancien Régime, the establishment of a constitutional monarchy, and eventually the French First Republic [Data: Reports (6, 106)].
- **Social Reforms**: Significant reforms included the abolition of feudal privileges, state control over the Catholic Church, and the promotion of individual rights [Data: Reports (6, 48, 49, 51, 57)].
- **Long-term Impact**: The Revolution had a lasting impact on French society and governance, influencing future democratic movements and shaping modern political discourse [Data: Reports (6, 2, 232, 235)].

This timeline encapsulates the major events and transformations during the French Revolution, highlighting its profound and lasting impact on France and beyond.

In [10]:
import yaml

if not os.path.exists('data/graphrag'):
  !python -m graphrag.index --init --root data/graphrag

with open('data/graphrag/settings.yaml', 'r') as f:
  settings_yaml = yaml.load(f, Loader=yaml.FullLoader)
settings_yaml['llm']['model'] = GPT_4_O_MODEL_NAME
settings_yaml['llm']['api_key'] = AZURE_OPENAI_API_KEY if is_azure else OPENAI_API_KEY
settings_yaml['llm']['type'] = 'azure_openai_chat' if is_azure else 'openai_chat'
settings_yaml['embeddings']['llm']['api_key'] = AZURE_OPENAI_API_KEY if is_azure else OPENAI_API_KEY
settings_yaml['embeddings']['llm']['type'] = 'azure_openai_embedding' if is_azure else 'openai_embedding'
settings_yaml['embeddings']['llm']['model'] = TEXT_EMBEDDING_3_LARGE_MODEL_NAME
if is_azure:
  settings_yaml['llm']['api_version'] = AZURE_OPENAI_API_VERSION
  settings_yaml['llm']['deployment_name'] = GPT_4_O_MODEL_NAME
  settings_yaml['llm']['api_base'] = AZURE_OPENAI_ENDPOINT
  settings_yaml['embeddings']['llm']['api_version'] = AZURE_OPENAI_API_VERSION
  settings_yaml['embeddings']['llm']['deployment_name'] = TEXT_EMBEDDING_3_LARGE_MODEL_NAME
  settings_yaml['embeddings']['llm']['api_base'] = AZURE_OPENAI_ENDPOINT

with open('data/graphrag/settings.yaml', 'w') as f:
  yaml.dump(settings_yaml, f)

if not os.path.exists('data/graphrag/input'):
  os.makedirs('data/graphrag/input')
  !cp data/french_revolution.md data/graphrag/input/french_revolution.txt
  !python -m graphrag.index --root ./data/graphrag

In [11]:
import subprocess

def ask_graph(query, response_type=DEFAULT_RESPONSE_TYPE):
  env = os.environ.copy() | {
    'GRAPHRAG_GLOBAL_SEARCH_MAX_TOKENS': str(DEFAULT_MAX_CONTEXT_TOKENS),
  }
  command = [
    'python', '-m', 'graphrag.query',
    '--root', './data/graphrag',
    '--method', 'local',
    '--response_type', response_type,
    query,
  ]
  output = subprocess.check_output(command, universal_newlines=True, env=env)
  return output.split('Search Response: ')[1]

In [12]:
from IPython.display import Markdown

result = ask_graph('Who was Robespierre and what was his role in the French revolution?')

Markdown(result)

# Maximilien Robespierre and His Role in the French Revolution

## Overview
Maximilien Robespierre was a central figure in the French Revolution, known for his radical views and significant influence within the revolutionary government. His actions and policies were pivotal during the Reign of Terror, a period marked by extreme political repression and mass executions.

## Key Roles and Actions
- **Leadership and Influence**:
  - Robespierre was a leading member of the Committee of Public Safety and a prominent leader within the Jacobins and the Montagnard faction [Data: Entities (52); Relationships (136, 263)].
  - He opposed property qualifications for voting and advocated for broader democratic participation [Data: Entities (52); Relationships (269, 272)].

- **Reign of Terror**:
  - Robespierre's policies were central to the Reign of Terror, which saw mass executions of perceived enemies of the revolution, including notable figures like Georges Danton and Hébert [Data: Entities (52, 186); Relationships (97, 266, 267)].
  - His radical stance and actions eventually led to his downfall, culminating in his arrest and execution on 28 July 1794 [Data: Entities (52); Relationships (275)].

- **Cult of the Supreme Being**:
  - Robespierre led the establishment of the Cult of the Supreme Being, a revolutionary cult that faced ridicule and opposition [Data: Entities (249); Relationships (268)].

- **Political Conflicts**:
  - He was involved in significant political conflicts, including opposition to the refractory clergy and the Convention, which eventually authorized his arrest [Data: Relationships (261, 275)].

Robespierre's radical vision and uncompromising stance made him a key architect of the revolution's most tumultuous period, but also led to his dramatic fall from power.


In [13]:
result = ask_graph('Timeline of the French revolution')

Markdown(result)

### Timeline of the French Revolution

The French Revolution was a decade-long period of significant political and social upheaval in France, marked by key events and transformations:

- **1789**: 
  - **May 5**: The Estates-General convenes in Versailles, marking the beginning of the revolution.
  - **June 17**: The Third Estate declares itself the National Assembly.
  - **July 14**: The Storming of the Bastille, a pivotal event symbolizing the uprising against the monarchy [Data: Relationships (0, 9, 59)].
  - **October 5-6**: The October 1789 March to Versailles by women demanding bread and political reforms [Data: Entities (397)].

- **1791**: 
  - **June 20-21**: Louis XVI attempts to flee Paris but is arrested in Varennes and brought back to Paris [Data: Relationships (81)].

- **1792**: 
  - **April 20**: France declares war on Austria, initiating the French Revolutionary Wars [Data: Entities (7)].
  - **August 10**: The Insurrection of 10 August 1792 leads to the fall of the monarchy [Data: Entities (14)].
  - **September 22**: The French First Republic is established, abolishing the monarchy [Data: Entities (5); Relationships (4)].

- **1793**: 
  - **January 21**: Execution of Louis XVI, marking a significant turning point [Data: Entities (3); Relationships (2)].
  - **September 5**: The Reign of Terror begins, characterized by extreme political repression and mass executions [Data: Entities (6); Relationships (5)].

- **1794**: 
  - **July 28**: Execution of Robespierre, ending the Reign of Terror [Data: Entities (6)].

- **1799**: 
  - **November 9**: The coup of 18 Brumaire leads to the establishment of the French Consulate, marking the end of the French Revolution [Data: Entities (7); Relationships (7)].

This timeline highlights the major events that defined the French Revolution, leading to profound changes in France's political and social structures.


In [16]:
QUESTIONS = [
  {
    'question': 'When did the French Revolution officially begin and end, and how long did it last?',
    'response_type': 'One short sentence (max 23 tokens)',
    'answer': 'The French Revolution began on 5 May 1789 and ended on 9 November 1799, lasting 10 years, 6 months, and 4 days.'
  },
  {
    'question': 'What were the key outcomes of the French Revolution ?',
    'response_type': 'List of 7-10 bullet points',
    'answer': ''
  },
  {
    'question': 'How did the financial and political crisis contribute to the calling of the Estates-General in 1789?',
    'response_type': DEFAULT_RESPONSE_TYPE,
    'answer': ''
  },
  {
    'question': 'What role did the Enlightenment and previous revolutions play in shaping the French Revolution?',
    'response_type': DEFAULT_RESPONSE_TYPE,
    'answer': ''
  },
  {
    'question': 'Analyze how the various social classes in France were affected by the Revolution and the policies implemented, such as the Civil Constitution of the Clergy and the abolition of feudal dues.',
    'response_type': DEFAULT_RESPONSE_TYPE,
    'answer': ''
  },
]

for question in QUESTIONS:
  question['answer_graph'] = ask_graph(question['question'], question['response_type'])
  question['answer_embeddings'] = ask_embeddings(question['question'], question['response_type'])

In [17]:
from pprint import pprint
pprint(QUESTIONS[0])

{'answer': 'The French Revolution began on 5 May 1789 and ended on 9 November '
           '1799, lasting 10 years, 6 months, and 4 days.',
 'answer_embeddings': 'The French Revolution officially began on 5 May 1789 '
                      'with the Estates General and ended on 9 November 1799 '
                      'with the coup of 18 Brumaire, which led to the '
                      'formation of the French Consulate. It lasted for 10 '
                      'years, 6 months, and 4 days [Data: Reports (1, 2, 3, 5, '
                      '7)].',
 'answer_graph': 'The French Revolution officially began on May 5, 1789, with '
                 'the convening of the Estates General, and ended on November '
                 '9, 1799, with the coup of 18 Brumaire, which established the '
                 'French Consulate. The revolution lasted for 10 years, 6 '
                 'months, and 4 days [Data: French Revolution and Its Key '
                 'Entities (88); French Revolution